In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

In [ ]:
import os
print(os.getcwd())

from torchrbpnet.data import tfrecord_to_dataloader, dummy_dataloader

dataloader = tfrecord_to_dataloader('../example/head.tfrecord')

In [ ]:
# %%
import torch
import torch.nn as nn
import pytorch_lightning as pl

import torchmetrics
from torchrbpnet.losses import MultinomialNLLLossFromLogits
from torchrbpnet.metrics import BatchedPCC, MultinomialNLLFromLogits
from torchrbpnet.networks import MultiRBPNet

# %%
class BatchIdx(torchmetrics.MeanMetric):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def update(self, batch_idx: torch.Tensor, *args, **kwargs):
        # update (i.e. take mean)
        super().update(batch_idx)

# %%
# class Model(pl.LightningModule):
#     def __init__(self, network):
#         super().__init__()
#         self.network = network
#         self.loss_fn = MultinomialNLLLossFromLogits()
#         self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()}) 
#         self.example_input_array = torch.rand(2, 4, 101)
#         self.test_metric = torchmetrics.MeanMetric() #BatchedPCC()

#         print(self.hparams)
    
#     def forward(self, *args, **kwargs):
#         return self.network(*args, **kwargs)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         return optimizer

#     def training_step(self, batch, batch_idx, **kwargs):
#         x, y = batch
#         y_pred = self.forward(x)
#         loss = self.loss_fn(y, y_pred, dim=-2)
#         self.compute_and_log_metics(y_pred, y, partition='train')
#         return loss
    
#     def training_epoch_end(self, *args, **kwargs):
#         self._reset_metrics()

#     def validation_epoch_end(self, *args, **kwargs):
#         self._reset_metrics()

#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_pred = self.forward(x)
#         self.compute_and_log_metics(y_pred, y, partition='val')
    
#     def compute_and_log_metics(self, y_pred, y, partition=None):
#         on_step = False
#         if partition == 'train':
#             on_step = True

#         for name, metric in self.metrics.items():
#             metric(y_pred, y)
#             self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
#     def _reset_metrics(self):
#         for metric in self.metrics.values():
#             metric.reset()

class Model(pl.LightningModule):
    def __init__(self, network, _example_input_shape):
        super().__init__()
        self.network = network
        self.loss_fn = MultinomialNLLLossFromLogits()
        # self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()}) # This has to be wrapped in a nn.ModuleDict (otherwise .to_device has to be called manually on metrics)
        self.metrics = nn.ModuleDict({'loss': MultinomialNLLFromLogits(), 'pcc': BatchedPCC()})
        # self.example_input_array = torch.rand(*_example_input_shape)
    
    def forward(self, *args, **kwargs):
        return self.network(*args, **kwargs)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx, **kwargs):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.loss_fn(y, y_pred, dim=-2)
        self.compute_and_log_metics(y_pred, y, partition='train')
        return loss
    
    def training_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_epoch_end(self, *args, **kwargs):
        self._reset_metrics()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        self.compute_and_log_metics(y_pred, y, partition='val')
    
    def compute_and_log_metics(self, y_pred, y, partition=None):
        on_step = False
        if partition == 'train':
            on_step = True

        for name, metric in self.metrics.items():
            metric(y_pred, y)
            self.log(f'{partition}/{name}', metric.compute(), on_step=on_step, on_epoch=True, prog_bar=False)
    
    def _reset_metrics(self):
        for metric in self.metrics.values():
            metric.reset()

model = Model(network=MultiRBPNet(n_tasks=223), _example_input_shape=next(iter(dataloader))[0].shape)

In [ ]:
print(next(iter(dataloader))[1].shape)
print(model(next(iter(dataloader))[0]).shape)

In [ ]:
import datetime
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, RichProgressBar

root_log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
loggers = [
    pl_loggers.TensorBoardLogger(root_log_dir+'/tensorboard', name='', version='', log_graph=True),
    # pl_loggers.CSVLogger(root_log_dir+'/tensorboard', name='', version=''),
]

checkpoint_callback = ModelCheckpoint(dirpath=f'{root_log_dir}/checkpoints', every_n_epochs=1, save_last=True)

early_stop_callback = EarlyStopping(monitor="val/loss", min_delta=0.00, patience=3, verbose=False, mode="min")

bar = RichProgressBar()

trainer = pl.Trainer(default_root_dir=root_log_dir, max_epochs=2, logger=loggers, callbacks=[checkpoint_callback, early_stop_callback])
trainer.fit(model=model, train_dataloaders=dataloader, val_dataloaders=dataloader)
# torch.save(model, 'model.pt')

In [ ]:
model

In [ ]:
torch.save(model, 'test.pt')